# Walk through of useful Tensor Operations
1. Tensor Reshaping (Mostly used)
2. Initialization of a Tensor
3. Tensor Mathematical Operations and Comparison
4. Tensor Indexing


But also other things such as setting the device (GPU/CPU) and converting
between different types (int, float etc) and how to convert a tensor to an
numpy array and vice-versa.

In [1]:
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # Cuda to run on GPU!

In [3]:
my_tensor = torch.tensor(
    [[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device=device, requires_grad=True
)

requires_grad就可以自动求导

In [4]:
device

'cuda'

In [5]:
my_tensor

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0', requires_grad=True)

In [6]:
my_tensor.dtype

torch.float32

In [7]:
my_tensor.device

device(type='cuda', index=0)

In [8]:
my_tensor.shape

torch.Size([2, 3])

In [9]:
my_tensor.requires_grad

True

# Tensor Reshaping

In [10]:
x = torch.arange(9)
# Let's say we want to reshape it to be 3x3

print("x : \n",x)

x_3x3 = x.view(3, 3)
print("x_3x3 :\n",x_3x3)

x_3x3 = x.reshape(3, 3)
print("x_3x3 :\n",x_3x3)

x : 
 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
x_3x3 :
 tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
x_3x3 :
 tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


In [11]:
# We can also do (view and reshape are very similar)
# and the differences are in simple terms (I'm no expert at this),
# is that view acts on contiguous tensors meaning if the
# tensor is stored contiguously in memory or not, whereas
# for reshape it doesn't matter because it will copy the
# tensor to make it contiguously stored, which might come
# with some performance loss.
# If we for example do:
y = x_3x3.t()
print(
    y.is_contiguous()
)  # This will return False and if we try to use view now, it won't work!
# y.view(9) would cause an error, reshape however won't

# This is because in memory it was stored [0, 1, 2, ... 8], whereas now it's [0, 3, 6, 1, 4, 7, 2, 5, 8]
# The jump is no longer 1 in memory for one element jump (matrices are stored as a contiguous block, and
# using pointers to construct these matrices). This is a bit complicated and I need to explore this more
# as well, at least you know it's a problem to be cautious of! A solution is to do the following
print(y.contiguous().view(9))  # Calling .contiguous() before view and it works

False
tensor([0, 3, 6, 1, 4, 7, 2, 5, 8])


# add two tensors along some axis

In [12]:
x1 = torch.arange(1, 25)
x1 = x1.reshape(2,3,4)

x2 = x1 * 2
print(torch.cat((x1, x2), dim=0).shape)  # Shape: 4x5
print(torch.cat((x1, x2), dim=1).shape)  # Shape 2x10

torch.Size([4, 3, 4])
torch.Size([2, 6, 4])


In [13]:
# Let's say we want to unroll x1 into one long vector with 10 elements, we can do:
z = x1.view(-1)  # And -1 will unroll everything
z

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24])

In [14]:
# If we instead have an additional dimension and we wish to keep those as is we can do:
batch = 64
x = torch.rand((batch, 2, 5))
z = x.view(
    batch, -1
)  # And z.shape would be 64x10, this is very useful stuff and is used all the time

z.shape

torch.Size([64, 10])

# change a tensor's dimension for A\*B\*C TO A\*C\*B

In [15]:
# Let's say we want to switch x axis so that instead of 64x2x5 we have 64x5x2
# I.e we want dimension 0 to stay, dimension 1 to become dimension 2, dimension 2 to become dimension 1
# Basically you tell permute where you want the new dimensions to be, torch.transpose is a special case
# of permute (why?)
print(x.shape)
z = x.permute(0, 2, 1)
print(z.shape)

torch.Size([64, 2, 5])
torch.Size([64, 5, 2])


# Split a tensor to many tensors

In [16]:
# Splits x last dimension into chunks of 2 (since 5 is not integer div by 2) the last dimension
# will be smaller, so it will split it into two tensors: 64x2x3 and 64x2x2
z = torch.chunk(x, chunks=3, dim=2)
print(z[0].shape)
print(z[1].shape)
print(z[2].shape)

torch.Size([64, 2, 2])
torch.Size([64, 2, 2])
torch.Size([64, 2, 1])


# add a trivial dimention 10 -> 10\*1 or 1\*10

In [17]:
x = torch.arange(
    10
)  # Shape is [10], let's say we want to add an additional so we have 1x10
print(x.unsqueeze(0).shape)  # 1x10
print(x.unsqueeze(1).shape)  # 10x1


torch.Size([1, 10])
torch.Size([10, 1])


# remove those trivial dim

In [18]:
# Let's say we have x which is 1x1x10 and we want to remove a dim so we have 1x10
x = torch.arange(10).unsqueeze(0).unsqueeze(1)
print("x.shape : ",x.shape) 
z = x.squeeze(0)
print("z.shape : ",z.shape)
z = x.squeeze(0).squeeze(0)
print("z.shape : ",z.shape)

x.shape :  torch.Size([1, 1, 10])
z.shape :  torch.Size([1, 10])
z.shape :  torch.Size([10])


# initializing tensors
# set initial values 0,1 random so on

In [19]:
x = torch.empty(size=(3, 3))

In [20]:
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [21]:
x = torch.empty(size=(3, 3))  # Tensor of shape 3x3 with uninitialized data
x = torch.zeros((3, 3))  # Tensor of shape 3x3 with values of 0
x = torch.rand(
    (3, 3)
)  # Tensor of shape 3x3 with values from uniform distribution in interval [0,1)
x = torch.ones((3, 3))  # Tensor of shape 3x3 with values of 1
x = torch.eye(5, 5)  # Returns Identity Matrix I, (I <-> Eye), matrix of shape 2x3
x = torch.arange(
    start=0, end=5, step=1
)  # Tensor [0, 1, 2, 3, 4], note, can also do: torch.arange(11)
x = torch.linspace(start=0.1, end=1, steps=10)  # x = [0.1, 0.2, ..., 1]
x = torch.empty(size=(1, 5)).normal_(
    mean=0, std=1
)  # Normally distributed with mean=0, std=1
x = torch.empty(size=(1, 5)).uniform_(
    0, 1
)  # Values from a uniform distribution low=0, high=1
x = torch.diag(torch.ones(3))  # Diagonal matrix of shape 3x3

# convert to different data types (int64, float, double) mostly used

In [22]:
tensor = torch.arange(4)  # [0, 1, 2, 3] Initialized as int64 by default
print("Converted int16\n {tensor.short()} 1 if nonzero")
    
print(f"Converted int16\n {tensor.short()} \n")  # Converted to int16

print(f"Converted float16\n {tensor.half()} \n")  # Converted to float16

print(f"Converted float64\n {tensor.double()}")  # Converted to float64

Converted int16
 {tensor.short()} 1 if nonzero
Converted int16
 tensor([0, 1, 2, 3], dtype=torch.int16) 

Converted float16
 tensor([0., 1., 2., 3.], dtype=torch.float16) 

Converted float64
 tensor([0., 1., 2., 3.], dtype=torch.float64)


In [23]:
print(
    f"Converted float32\n {tensor.float()}"
)  # Converted to float32 (This one is very important, used super often)

print(
    f"Converted int64\n {tensor.long()}"
)  # Converted to int64 (This one is very important, used super often)


Converted float32
 tensor([0., 1., 2., 3.])
Converted int64
 tensor([0, 1, 2, 3])


# switch between numpy ad tensor

In [24]:
import numpy as np

In [25]:
np_array = np.zeros((5, 5))

In [26]:
tensor = torch.from_numpy(np_array) ## 只能转换之后再改类型
tensor = tensor.float()
tensor.dtype

torch.float32

In [27]:
np_array = (
    tensor.numpy()
)
np_array

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [28]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([9, 8, 7])

# -- Addition --
z = x + y

# -- Subtraction --
z = x - y  # We can do similarly as the preferred way of addition

# -- Division (A bit clunky) --
z = torch.true_divide(x, y)  # Will do element wise division if of equal shape

## A.add_(B) will change A ， A.add(B) will not

In [29]:
t = torch.zeros(3)
t.add_(x)
t

tensor([1., 2., 3.])

In [30]:
t = torch.zeros(3)
t.add(x)
t

tensor([0., 0., 0.])

In [31]:
z = x.pow(2)  # z = [1, 4, 9]
z = x ** 2  # z = [1, 4, 9]

In [32]:
# -- Simple Comparison --
z = x > 0  # Returns [True, True, True]
z = x < 0  # Returns [False, False, False]

# Matrix Multiplication A.mm(B)

In [33]:
x1 = torch.rand((2, 5))
x2 = torch.rand((5, 3))
x3 = torch.mm(x1, x2)  # Matrix multiplication of x1 and x2, out shape: 2x3
x3 = x1.mm(x2)  # Similar as line above

# Matrix Exponentiation

In [34]:
matrix_exp = torch.rand(5, 5)
print(
    matrix_exp.matrix_power(3)
)  # is same as matrix_exp (mm) matrix_exp (mm) matrix_exp

tensor([[2.7060, 4.8615, 1.9560, 3.4318, 2.8762],
        [3.9800, 7.1649, 2.7901, 4.9068, 4.1118],
        [2.8601, 5.1823, 2.0280, 3.6318, 2.9458],
        [1.8719, 3.4413, 1.3883, 2.4582, 2.0266],
        [2.7577, 5.0379, 2.0146, 3.5526, 2.9943]])


# Element wise Multiplication

In [35]:
z = x * y  # z = [9, 16, 21] = [1*9, 2*8, 3*7]

# Dot product only for 1-D tensor, for matrix have to reshape

In [36]:
z = torch.dot(x, y)  # Dot product, in this case z = 1*9 + 2*8 + 3*7
print(z)

tensor(46)


# Batch Matrix Multiplication

In [37]:
batch = 32
n = 10
m = 20
p = 30
tensor1 = torch.rand((batch, n, m))
tensor2 = torch.rand((batch, m, p))
out_bmm = torch.bmm(tensor1, tensor2)  # Will be shape: (b x n x p)

# Other useful tensor operations

In [38]:
sum_x = torch.sum(
    x, dim=0
)  # Sum of x across dim=0 (which is the only dim in our case), sum_x = 6

values, indices = torch.max(x, dim=0)  # Can also do x.max(dim=0)

values, indices = torch.min(x, dim=0)  # Can also do x.min(dim=0)

abs_x = torch.abs(x)  # Returns x where abs function has been applied to every element

z = torch.argmax(x, dim=0)  # Gets index of the maximum value

z = torch.argmin(x, dim=0)  # Gets index of the minimum value

mean_x = torch.mean(x.float(), dim=0)  # mean requires x to be float

z = torch.eq(x, y)  # Element wise comparison, in this case z = [False, False, False]

z = torch.clamp(x, min=0)
# All values < 0 set to 0 and values > 0 unchanged (this is exactly ReLU function)

# sort the elements

In [39]:
sorted_y, indices = torch.sort(y, dim=0, descending=False)
print('y : ',y)
print('sorted_y : ', sorted_y )
print("indices : " , indices )

y :  tensor([9, 8, 7])
sorted_y :  tensor([7, 8, 9])
indices :  tensor([2, 1, 0])


In [40]:
# If you want to values over max_val to be clamped, do torch.clamp(x, min=min_val, max=max_val)
x = torch.tensor([1, 0, 1, 1, 1], dtype=torch.bool)  # True/False values
print(x)
z = torch.any(x)  # will return True, can also do x.any() instead of torch.any(x)
print(z)
z = torch.all(
    x
)  # will return False (since not all are True), can also do x.all() instead of torch.all()
print(z)

tensor([ True, False,  True,  True,  True])
tensor(True)
tensor(False)


# Tensor Indexing  

In [41]:
batch_size = 10
features = 25
x = torch.rand((batch_size, features))

In [42]:
# Get first examples features
print(x[0].shape)  # shape [25], this is same as doing x[0,:]

torch.Size([25])


In [43]:
# Get the first feature for all examples
print(x[:, 0].shape)  # shape [10]

torch.Size([10])


In [44]:
# For example: Want to access third example in the batch and the first ten features
print(x[2, 0:10].shape)  # shape: [10]

torch.Size([10])


In [45]:
# For example we can use this to, assign certain elements
x[0, 0] = 100

# Index with a list or lists, same in numpy

In [46]:
x = torch.arange(10)
indices = [2, 5, 8]
print(x[indices])  # x[indices] = [2, 5, 8]

tensor([2, 5, 8])


In [47]:
x = torch.rand((3, 5))
rows = torch.tensor([1, 0])
cols = torch.tensor([4, 0])
print(x)
print(x[rows, cols])  # Gets second row fifth column and first row first column

tensor([[0.5067, 0.7953, 0.5864, 0.6685, 0.8093],
        [0.1426, 0.2775, 0.5886, 0.9847, 0.7986],
        [0.7934, 0.0702, 0.8076, 0.7612, 0.5713]])
tensor([0.7986, 0.5067])


# select sublist according to value

In [48]:
x = torch.arange(10)
print(x)
print(x[(x < 2) | (x > 8)])  # will be [0, 1, 9]
print(x[x.remainder(2) == 0])  # depend on value not position
print(
    torch.where(x > 5, x, x * 2)
)  # gives [0, 2, 4, 6, 8, 10, 6, 7, 8, 9], all values x > 5 yield x, else x*2
x = torch.tensor([0, 0, 1, 2, 2, 3, 4]).unique()  # x = [0, 1, 2, 3, 4]

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0, 1, 9])
tensor([0, 2, 4, 6, 8])
tensor([ 0,  2,  4,  6,  8, 10,  6,  7,  8,  9])


In [49]:
print(
    'dimension' , x.ndimension()
)  # The number of dimensions, in this case 1. if x.shape is 5x5x5 ndim would be 3

x = torch.rand((3, 5))
print(
    'The number of elements' , x.numel()
)  # The number of elements in x (in this case it's trivial because it's just a vector)

dimension 1
The number of elements 15
